In [2]:
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from scipy.stats import ttest_1samp

# Importing the data

In [3]:
# Importing data
df = pd.read_stata("bwght2.dta")
print(df.head())
print(df.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'bwght2.dta'

In [ ]:
df.describe()

# Checking for null values

In [ ]:
df.isna().any()

# Q1a: What is the average birth weight for the babies in this sample?

In [ ]:
df["bwght"].mean()

# Q1b: Find the median birth weight

In [ ]:
df["bwght"].median()

# Q1c: Draw the empirical histogram for the birth weight

In [ ]:
df["bwght"].plot(kind = "hist", bins = 32)

# Q1d: What is the Interpretation of beta_3? Find out the OLS estimate.

In [ ]:
# Removing null values
# df = df.dropna()


# Setting up the Regression Model

X = df[["npvis", "npvissq", "cigs", "male"]]
X = sm.add_constant(X)
y = df["bwght"]


model = sm.OLS(y, X, missing = "drop")
results = model.fit()
print(results.summary())

# Beta_3's estimate is -9.6390. The interpretation of beta_3 is that for every additional cigarette smoked per day on average during preganncy, the birth weigh  decreases by -9.6390grams on average, ceteris paribus.


# Question 1e) Test the null hypothesis that beta_3 = -10 against the alternative beta_3 != -10 at 5% significance level

In [ ]:
results.conf_int(alpha = 0.05, cols = None)

# Fail to reject the null hypothesis that beta_3 = -10 at alpha = 0.05, since -10 falls within the 95% confidence interval

# Question 1f) Test the hypothesis that newborn boys weigh by 100grams more than newborn girls at 95% confidence interval. Write down the null and alternative hypothesis.

- H_0: newborn boys weigh 100grams compared to newborn girls (beta_4 = 100)

- H_A: newborn boys weigh less than or more than 100 grams compared to newborn girls (beta_4 != 100)

In [ ]:
hypothesis = "male = 100"

t_test = results.t_test(hypothesis)
print(t_test)
print(t_test.pvalue)

# p value = 0.6976 > 0.05, therefore we fail to reject the null hypothesis at alpha = 0.05

# Q1g) What is the partial effect of the number of prenatal doctor visits (npvis) on birth weight?

In [ ]:
# Manually calculate the partial derivative

# Q1h) At what point does one more time of prenatal doctor visits (npvis) reduce the birth weight of the baby? Explain the economic intuition of your result.

In [ ]:
# Set the partial derivative to 0 and solve, since that is the threshold to find the point of diminishing returns

# More doctor visits could imply a health issue, which would affect the birth weight of the newborn

# Q1i) Is the homoskedasticity assumption met? If not, how would you solve the problem?

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white

white_test = het_white(results.resid, results.model.exog)
bp_test = het_breuschpagan(results.resid, results.model.exog)

labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]
print(dict(zip(labels, bp_test)))
print(dict(zip(labels, white_test)))

# The null hypothesis is that there is homoskedasticity (p > 0.05)

# If p > 0.05, there is homoskedasticity. In this case, p values are both > 0.05, meaning that there is homoskedasticity. 

# If not, you can always use robust standard error with large sample sizes

## THE ABOVE OUTPUT IS WRONG... there is supposedly heteroskedastic error

# Q1j) Do you think that the zero conditional mean assumption holds in this regression model? Why or why not?

In [ ]:
# Yes... It is very likely that an excluded variable is highly correlated with the indepedent variables. For example, one of the error term variables could be the welath of the family, which easily correlates with the number of doctor visits.